In [2]:
# determine resolution limits of 1 kHz, 16 kHz, 50 kHz files for different decimations, each event

from rss_ringoccs_local_tools import *

In [135]:

# for simplicity, use CORSS_8001 files

dlp_range = [60000,150000]
CORSS_file_list = global_path_to_tables+'CORSS_8001_TAU_filepaths.txt'
with open(CORSS_file_list,'r') as f:
    events_ = f.read().splitlines()
events = []
for event in events_:
    if '#' not in event:
        events.append(event)
freqskHz = np.array([1,2,4,8,16,25,50])
res_desired=np.array([500,250,100,50,25])
rev_prev=-1
direc_prev = ''

table = '../tables/resolution_limits.txt'
with open (table,'w') as file:
    print('Min. res_km*resolution factor(m)     BW (kHz) for desired resolution',file=file)
    print('....................................................................',file=file)
    print('Rev   Dir   1kHz   4kHz  16kHz 50kHz   500m   250m   100m  50m  25m',file=file)
    print('--------------------------------------------------------------------',file=file)
    for i,event in enumerate(events):
        # if i != 0:
        #     continue
        taufile = global_path_to_data+event
        direc = taufile.split(os.sep)[-2][-1]
        rev_num = taufile.split(os.sep)[-4][3:]
        geofile = taufile.replace('TAU_01KM','GEO')
    
        rdot = np.gradient(rkm,spm)
        spm,rkm = np.loadtxt(geofile,delimiter=',',usecols=[1,3]).T
        rdot = np.gradient(rkm,spm)
        if direc == 'I':
            L= np.where((rdot <0) & (rkm >= dlp_range[0]) & (rkm <= dlp_range[1]))[0]
        elif direc =='E':
            L= np.where((rdot >0) & (rkm >= dlp_range[0]) & (rkm <= dlp_range[1]))[0]
        else:
            raise Exception('Invalid direction ',direc)
        if len(L)>0:
            LL = np.argmax(np.abs(rdot[L]))
            l = L[LL]
            max_rdot = np.max(np.abs(rdot[L]))
            res_m = np.ceil(2*max_rdot/freqskHz).astype(int)
            freqs_required = np.zeros(len(res_desired),dtype = int)
            for ii,res in enumerate(res_desired):
                mm = np.min(np.where(res_m<=res))
                freqs_required[ii] = freqskHz[mm]
            f = freqs_required # shorthand
            if rev_num != rev_prev or direc != direc_prev:
                if rev_num != rev_prev and i !=0:
                    print(' ',file=file)
                print('Rev'+rev_num,direc,f'{res_m[0]:6d}{res_m[2]:7d}{res_m[4]:7d}{res_m[6]:6d}{f[0]:6d}{f[1]:7d}{f[2]:7d}{f[3]:6d}{f[4]:5d}',file=file)
            rev_prev = rev_num
            direc_prev = direc
with open(table,'r') as file:
    lines = [line.strip() for line in file]

for line in lines:
    print(line)

Min. res_km*resolution factor(m)     BW (kHz) for desired resolution
....................................................................
Rev   Dir   1kHz   4kHz  16kHz 50kHz   500m   250m   100m  50m  25m
--------------------------------------------------------------------
Rev007 E     27      7      2     1     1      1      1     1    2
Rev007 I     26      7      2     1     1      1      1     1    2

Rev008 E     27      7      2     1     1      1      1     1    2
Rev008 I     27      7      2     1     1      1      1     1    2

Rev009 E     27      7      2     1     1      1      1     1    2

Rev010 E     27      7      2     1     1      1      1     1    2
Rev010 I     27      7      2     1     1      1      1     1    2

Rev011 E     27      7      2     1     1      1      1     1    2

Rev012 E     26      7      2     1     1      1      1     1    2
Rev012 I     26      7      2     1     1      1      1     1    2

Rev013 E     25      7      2     1     1      1 

In [79]:
res_m

array([8, 1, 1])